# Table of Contents
 <p><div class="lev2 toc-item"><a href="#学习心得" data-toc-modified-id="学习心得-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>学习心得</a></div><div class="lev1 toc-item"><a href="#神经网络翻译模型" data-toc-modified-id="神经网络翻译模型-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>神经网络翻译模型</a></div><div class="lev2 toc-item"><a href="#梯度消散（Gradient-Vanish）问题" data-toc-modified-id="梯度消散（Gradient-Vanish）问题-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>梯度消散（Gradient Vanish）问题</a></div><div class="lev2 toc-item"><a href="#翻译问题" data-toc-modified-id="翻译问题-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>翻译问题</a></div><div class="lev3 toc-item"><a href="#传统机器翻译（IBM-Model）" data-toc-modified-id="传统机器翻译（IBM-Model）-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>传统机器翻译（IBM Model）</a></div><div class="lev3 toc-item"><a href="#神经机器翻译" data-toc-modified-id="神经机器翻译-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>神经机器翻译</a></div><div class="lev3 toc-item"><a href="#神经机器翻译理论模型" data-toc-modified-id="神经机器翻译理论模型-123"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>神经机器翻译理论模型</a></div><div class="lev2 toc-item"><a href="#Tensorflow-seq2seq" data-toc-modified-id="Tensorflow-seq2seq-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Tensorflow seq2seq</a></div><div class="lev2 toc-item"><a href="#代码学习" data-toc-modified-id="代码学习-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>代码学习</a></div><div class="lev2 toc-item"><a href="#作业" data-toc-modified-id="作业-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>作业</a></div>

## 学习心得


- 理解模型的基本结构和数学原理，掌握CNN，RNN，Word Embedding， Gradient descent等基本原理，对后续的学习非常有帮助，否则会知其然不知其所以然。配合阅读一些论文效果会更好，目前我看的论文数量还不多，后面有时间要多看论文。开课前读过Michael Nielsen的书[Neural Networks and Deep Learning](Neural Networks and Deep Learning), 很有帮助。  

- 掌握tensorflow对应API的使用，重点是理解各个参数的数据维度。学习方法是看官方API文档，同时结合别人写的例子代码来看，如果对参数结构不明白的，最好运行例子代码，把每个参数的shape打印出来，会加快理解。  

- 准备训练数据，第一步是要原始数据做预处理，然后按照模型API的要求生成训练数据集。这部分代码工作量不小，不过慢慢可以积累一些可以复用的代码，可参考tensorflow官方教程中的一些数据处理代码。  

- 训练模型，需要尝试调整超参数集，估计训练时间，尽快实现模型收敛。对训练时间长的模型，训练结果要保存下来，以后可以继续训练或用来做预测，否则下次又要重头开始训练。

# 神经网络翻译模型

## 梯度消散（Gradient Vanish）问题  

- 为什么早期不直接用深层神经网络？ 
  - 数据不够
  - 计算资源不够
  - 梯度消散问题
- ReLU
  - 梯度（导数）要么是 0 要么是 1
  - 对于小于 0 的部分，置为了 0：Sparse activation；可能更符合生物神经网络特性，低于一定阈值就不传输了；后面的矩阵更稀疏也可能正好是我们需要的
- 小于 4 层 5 层，tanh 足够了；不是绝对的
- 如果层数加深收敛的不够好，可以使用 ReLU
- 或者层数不多但希望得到稀疏激活的情况，也可以使用 ReLU
- 在计算量允许的情况下，不断加深层数，直到验证集错误率不再改进

## 翻译问题

- Source -> Target
  - 中文 -> 英文
  - 微博 -> 微博评论
  - 邮件 -> 回复
  - 命令 -> （助理的）回复

- Target 会根据 Source 的内容做一些反应

### 传统机器翻译（IBM Model）  

- 平行语料，词对齐  


### 神经机器翻译

- 和人翻译有些类似
- 过完一句英文，拿到一整句的词向量，根据词向量再翻译出每个词
  - 英文 -> 句子向量：Encode，编码器
  - 句子向量 -> Target：Decode，解码器
- Sequence to Sequence (seq2seq)
  - 假设模型已经训练好
  - 用最后状态的输出作为 ContextVector（包含隐藏的语义）
  - 一个 RNN Cell 接收三个输入：State，ContextVector，上一个时间的词向量（刚开始可以给个 start token）
  - 输出一个 Output，接 Softmax，得出 y1
  - 进入下一个 Cell，接收：State，ContextVector，y1
  - 只看 y2 的问题时，y1 的 softmax 是没有用的，但是 y1 本身的问题也要考虑，所以有多个分类问题综合考虑，一起产生梯度更新参数的过程
  - 直到遇到 end token（softmax 中 end 的概率最大时）
- 参数复用，会有好几个梯度，但都算到同一个 RNN 矩阵上
  - 每个 RNN 拿到的梯度不一样，但最后都更新到同一个矩阵上


### 神经机器翻译理论模型

- $P(y_1 | C)$
- $P(y_2 | y_1, C)$
- $P(y_3 | y_1, y_2, C)$
- $P(y_1, y_2,...,y_n | C) = \prod_i P(y_i | y_1, y_2,...,y_{i-1}, C)$

Language Model + Context  

[官方文档](http://www.tensorflow.org/tutorials/seq2seq)

10:58 有间断

## Tensorflow seq2seq

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.contrib.legacy_seq2seq import basic_rnn_seq2seq, embedding_rnn_seq2seq, sequence_loss
from tensorflow.python.ops import variable_scope

In [3]:
vocab = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    '<GO>': 6,
    '<EOS>': 7
}

reverse_vocab = dict([(v, k) for (k, v) in vocab.iteritems()])

In [4]:
reverse_vocab

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: '<GO>', 7: '<EOS>'}

In [5]:
tf.reset_default_graph()

In [6]:
vocab

{'<EOS>': 7, '<GO>': 6, 'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5}

In [7]:
map(lambda _:[vocab[_]], ['A','B','C'])

[[0], [1], [2]]

In [8]:
encoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['A', 'B', 'C'])
decoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['<GO>', 'D', 'E', 'F', '<EOS>'])

In [9]:
encoder_inputs

[<tf.Tensor 'Const:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_1:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_2:0' shape=(1,) dtype=int32>]

In [10]:
decoder_inputs

[<tf.Tensor 'Const_3:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_4:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_5:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_6:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_7:0' shape=(1,) dtype=int32>]

In [11]:
cell = tf.contrib.rnn.BasicRNNCell(128)
num_encoder_symbols = 8
num_decoder_symbols = 8
embedding_size = 128

outputs, states = embedding_rnn_seq2seq(
    encoder_inputs, decoder_inputs, cell, 
    num_encoder_symbols, num_decoder_symbols, 
    embedding_size, output_projection=None, feed_previous=False)

In [12]:
weights = map(lambda _: tf.constant([_], dtype=tf.float32), [1, 1, 1, 1, 1])
loss = sequence_loss(outputs, decoder_inputs, weights)

In [13]:
weights

[<tf.Tensor 'Const_8:0' shape=(1,) dtype=float32>,
 <tf.Tensor 'Const_9:0' shape=(1,) dtype=float32>,
 <tf.Tensor 'Const_10:0' shape=(1,) dtype=float32>,
 <tf.Tensor 'Const_11:0' shape=(1,) dtype=float32>,
 <tf.Tensor 'Const_12:0' shape=(1,) dtype=float32>]

In [14]:
loss

<tf.Tensor 'sequence_loss/truediv:0' shape=() dtype=float32>

In [15]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [16]:
import numpy as np

In [17]:
with tf.Session() as sess:
    # Training
    sess.run(tf.global_variables_initializer())
#     for iteration in range(50):
#         sess.run(train_step)
#         print(sess.run(loss))
    
    # Decoding
    with variable_scope.variable_scope(variable_scope.get_variable_scope(), reuse=True):
        decode_decoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['<GO>', 'A', 'A', 'A', 'A'])
        outputs, states = embedding_rnn_seq2seq(
            encoder_inputs, decode_decoder_inputs, cell,
            num_encoder_symbols, num_decoder_symbols,
            embedding_size, output_projection=None,
            feed_previous=True)
        
        for o in outputs:
            m = np.argmax(o.eval(), axis=1)
            print(reverse_vocab[m[0]])

E
F
F
<EOS>
B


In [18]:
decode_decoder_inputs

[<tf.Tensor 'Const_13:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_14:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_15:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_16:0' shape=(1,) dtype=int32>,
 <tf.Tensor 'Const_17:0' shape=(1,) dtype=int32>]

In [29]:
with tf.Session() as sess:
    # Training
    sess.run(tf.global_variables_initializer())
    for iteration in range(50):
        sess.run(train_step)
        print(sess.run(loss))
    
    # Decoding
    # embedding 内建，reuse 是复用之前训练好的 embedding，需要在同一个 session 里面
    with variable_scope.variable_scope(variable_scope.get_variable_scope(), reuse=True):
        decode_decoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['<GO>', 'A', 'A', 'A', 'A'])
        outputs, states = embedding_rnn_seq2seq(
            encoder_inputs, decode_decoder_inputs, cell,
            num_encoder_symbols, num_decoder_symbols,
            embedding_size, output_projection=None,
            feed_previous=True)
        
        for o in outputs:
            m = np.argmax(o.eval(), axis=1)
            print(reverse_vocab[m[0]])

1.85723
1.64927
1.47348
1.32612
1.20205
1.09635
1.00502
0.92502
0.854147
0.790796
0.733791
0.682248
0.635484
0.592952
0.554203
0.518852
0.486567
0.457053
0.430047
0.405312
0.382632
0.361814
0.342682
0.325075
0.308851
0.29388
0.280044
0.267239
0.255369
0.24435
0.234104
0.224563
0.215664
0.207353
0.199579
0.192297
0.185466
0.179049
0.173013
0.167327
0.161966
0.156904
0.152118
0.147588
0.143296
0.139224
0.135358
0.131682
0.128185
0.124854
<GO>
D
E
F
<EOS>


## 代码学习

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.legacy_seq2seq import basic_rnn_seq2seq, embedding_rnn_seq2seq, sequence_loss
from tensorflow.python.ops import variable_scope

vocab = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    '<GO>': 6,
    '<EOS>': 7
}

reverse_vocab = dict([(v, k) for (k, v) in vocab.iteritems()])

tf.reset_default_graph()

encoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['A', 'B', 'C'])
decoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['<GO>', 'D', 'E', 'F', '<EOS>'])

cell = tf.contrib.rnn.BasicRNNCell(128)
num_encoder_symbols = 8
num_decoder_symbols = 8
embedding_size = 128

outputs, states = embedding_rnn_seq2seq(
    encoder_inputs, decoder_inputs, cell, 
    num_encoder_symbols, num_decoder_symbols, 
    embedding_size, output_projection=None, feed_previous=False)

weights = map(lambda _: tf.constant([_], dtype=tf.float32), [1, 1, 1, 1, 1])
loss = sequence_loss(outputs, decoder_inputs, weights)

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
    # Training
    sess.run(tf.global_variables_initializer())
    for iteration in range(50):
        sess.run(train_step)
        print(sess.run(loss))
        #print(sess.run(outputs))
    for o in outputs:
        m = np.argmax(o.eval(), axis=1)
        print(reverse_vocab[m[0]])
    
    # Decoding
    # embedding 内建，reuse 是复用之前训练好的 embedding，需要在同一个 session 里面
#     with variable_scope.variable_scope(variable_scope.get_variable_scope(), reuse=True):
#         decode_decoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['<GO>', 'A', 'A', 'A', 'A'])
#         outputs, states = embedding_rnn_seq2seq(
#             encoder_inputs, decode_decoder_inputs, cell,
#             num_encoder_symbols, num_decoder_symbols,
#             embedding_size, output_projection=None,
#             feed_previous=True)
        
#         for o in outputs:
#             m = np.argmax(o.eval(), axis=1)
#             print(reverse_vocab[m[0]])

1.65117
1.47759
1.32882
1.20154
1.09193
0.996611
0.912944
0.838945
0.77311
0.714281
0.66153
0.614101
0.571357
0.532755
0.497828
0.466168
0.437418
0.411262
0.387424
0.365659
0.34575
0.327505
0.310754
0.295346
0.281149
0.268042
0.25592
0.24469
0.234268
0.224578
0.215556
0.20714
0.199278
0.191921
0.185027
0.178558
0.172478
0.166757
0.161365
0.156278
0.151472
0.146926
0.142621
0.138539
0.134665
0.130984
0.127484
0.124151
0.120974
0.117945
<GO>
D
E
F
<EOS>


In [34]:
vocab

{'<EOS>': 7, '<GO>': 6, 'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5}

In [35]:
reverse_vocab

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: '<GO>', 7: '<EOS>'}

In [21]:
map(lambda _: ([vocab[_]]), ['<GO>', 'A', 'A', 'A', 'A'])

[[6], [0], [0], [0], [0]]

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.legacy_seq2seq import basic_rnn_seq2seq, embedding_rnn_seq2seq, sequence_loss
from tensorflow.python.ops import variable_scope

vocab = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    '<GO>': 6,
    '<EOS>': 7
}

reverse_vocab = dict([(v, k) for (k, v) in vocab.iteritems()])

tf.reset_default_graph()

encoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['A', 'B', 'C'])
decoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['<GO>', 'D', 'E', 'F', '<EOS>'])

cell = tf.contrib.rnn.BasicRNNCell(128)
num_encoder_symbols = 8
num_decoder_symbols = 8
embedding_size = 128

outputs, states = embedding_rnn_seq2seq(
    encoder_inputs, decoder_inputs, cell, 
    num_encoder_symbols, num_decoder_symbols, 
    embedding_size, output_projection=None, feed_previous=False)

weights = map(lambda _: tf.constant([_], dtype=tf.float32), [1, 1, 1, 1, 1])
loss = sequence_loss(outputs, decoder_inputs, weights)

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
    # Training
    sess.run(tf.global_variables_initializer())
    for iteration in range(50):
        sess.run(train_step)
        print(sess.run(loss))

    # Decoding
    # embedding 内建，reuse 是复用之前训练好的 embedding，需要在同一个 session 里面
    with variable_scope.variable_scope(variable_scope.get_variable_scope(), reuse=True):
        decode_decoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['<GO>', 'A', 'A', 'A', 'A'])
        outputs, states = embedding_rnn_seq2seq(
            encoder_inputs, decode_decoder_inputs, cell,
            num_encoder_symbols, num_decoder_symbols,
            embedding_size, output_projection=None,
            feed_previous=True)
        
        for o in outputs:
            m = np.argmax(o.eval(), axis=1)
            print(reverse_vocab[m[0]])

1.66484
1.49299
1.3445
1.21622
1.10478
1.00727
0.921296
0.845033
0.777043
0.716196
0.661582
0.612454
0.56818
0.528222
0.492107
0.459424
0.429807
0.402929
0.378501
0.356266
0.335993
0.317476
0.300534
0.285005
0.270743
0.257621
0.245526
0.234355
0.22402
0.214439
0.205543
0.197267
0.189555
0.182357
0.175627
0.169325
0.163414
0.157862
0.15264
0.147722
0.143082
0.138701
0.134557
0.130634
0.126916
0.123387
0.120035
0.116847
0.113811
0.110919
<GO>
D
E
F
<EOS>


- Tensorflow 中的 seq2seq（简化版本）
  - state 用 ContextVector 做初始化
  - input 不变（词向量）
  - encoder 和 decoder RNN 是同一个

## 作业

- 阅读 Tensorflow seq2seq 文档
- 阅读 Tensorflow 文档中 variable_scope 的部分，理解其语义及作用
- 阅读 Tensorflow 中 seq2seq.py 中 embedding_rnn_seq2seq, sequence_loss 相关代码
- 基于课程框架，使用给定平行语料（训练、测试）完成翻译模型，产生翻译结果
  - 进阶：学习并使用 embedding_attention_seq2seq 相关代码，配合 LSTMCell 或 GRUCell 改善模型效果